<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/chizu_000_task2_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2_classification"
    name = "chizu_000_task2_complete_all"

    # pseudo_base_model_name = "ioai2024japan/redrock_015_task2_finetune"
    base_model_name = "google-bert/bert-base-multilingual-uncased"
    base_tokenizer_name = "google-bert/bert-base-multilingual-uncased"

    # None -> training, otherwise -> load the model
    pretrained_model_name = "ioai2024japan/redrock_005_task2_pretrain_wandb"
    # tokenizer_name = "google-bert/bert-base-multilingual-uncased"
    num_classes = 5

    # training
    pretrain_epochs = 1
    classification_epochs = 30
    mlm_probability = 0.15

    scheduler='linear' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 1e-05

    # dataset
    max_length = 256

    # T4: 32
    # L4: 64
    train_batch_size = 32
    eval_batch_size = 32

    seed=42
    train=True

    pseudo_size = 60000
    pseudo_select_size = 1500

    if_wandb = False

# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [2]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0s
  !pip install evaluate==0.4.2
  !pip install accelerate -U

if importlib.util.find_spec('wandb') is None:
  !pip install wandb -q

import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F
import torch.cuda.amp as amp

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm
from transformers import DataCollatorForLanguageModeling

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler, BertForMaskedLM, BertTokenizer

from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

import evaluate

import wandb

from joblib import Parallel, delayed

from huggingface_hub import login

wandb.login(key=userdata.get('wandb_token'))
login(token=read_access_token)

brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': '90'
}

def transliterate_brahmi_to_devanagari(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text

f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output


wandb: Currently logged in as: asiatic-cheetah (asiatic-cheetah-a). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
def pred_pseudo(model, train_loader, device):
    model.eval()
    predictions_list = []
    confidences_list = []
    for batch in tqdm(train_loader):
        batch = to_device(batch, device)
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        confidences = F.softmax(logits, dim=-1)
        pred_confidence,_ = confidences.max(dim=-1)
        #pred_confidence,_ = pred_confidence_temp.max(dim=-1)
        # print(pred_confidence.shape)
        predictions = torch.argmax(logits, dim=-1)
        #predictions = torch.mode(predictions, dim=0).values
        # print(predictions.shape)
        predictions_list.extend(predictions.cpu().numpy())
        confidences_list.extend(pred_confidence.cpu().numpy())

    return predictions_list, confidences_list

In [4]:
def pseudo_get_data(raw_dataset, transform_raw, finetuned_model_path, device):
    pseudo_model = AutoModelForSequenceClassification.from_pretrained(
        finetuned_model_path, num_labels=CFG.num_classes
    ).cuda()

    pseudo_size = CFG.pseudo_size

    raw_eval_batch = raw_dataset["train"].select(range(0, pseudo_size))

    tokenized_eval_batch = raw_eval_batch.with_transform(transform_raw)

    raw_loader = DataLoader(
        tokenized_eval_batch,
        batch_size=CFG.train_batch_size * 2,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )

    predictions, confidences = pred_pseudo(pseudo_model, raw_loader, device)

    top_conf = np.argsort(confidences)[-CFG.pseudo_select_size:]

    selected_texts = [transliterate_brahmi_to_devanagari(raw_eval_batch[int(i)]["text"]) for i in top_conf]
    selected_labels = [predictions[i] for i in top_conf]

    pseudo_labeled_dataset = Dataset.from_dict({
        'text': selected_texts,
        'label': selected_labels
    })

    return pseudo_labeled_dataset, confidences

In [5]:
def train_tokenizer(raw_dataset):
    train_corpus = []

    num_cores = 8

    train_corpus = Parallel(n_jobs=num_cores)(
        delayed(transliterate_brahmi_to_devanagari)(text) for text in tqdm(raw_dataset['train']["text"])
    )

    # print(train_corpus[:5])

    old_tokenizer = AutoTokenizer.from_pretrained(CFG.base_tokenizer_name)

    tokenizer = old_tokenizer.train_new_from_iterator(train_corpus, old_tokenizer.vocab_size)

    return tokenizer

In [6]:
def train_one_epoch(model, scheduler, train_loader, optimizer, fp16=False):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_loader, dynamic_ncols=True)
    scaler = torch.cuda.amp.GradScaler()

    for step, batch in enumerate(progress_bar):
        batch = to_device(batch, "cuda")

        if fp16:
            with amp.autocast():
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    token_type_ids=batch["token_type_ids"],
                    labels=batch["labels"],
                )
                loss = outputs.loss

            # Scale loss for fp16 training
            scaler.scale(loss).backward()

            # Optimizer step with gradient scaling
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        else:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )
            loss = outputs.loss
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        text = f"step {step}, loss: {loss:.5f}"
        progress_bar.set_description(text)

def evaluate_model(model, eval_loader):
    model.eval()
    predictions = []
    labels = []
    for batch in eval_loader:
        batch = to_device(batch, "cuda")
        with torch.no_grad():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )

        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1)
        predictions.append(prediction.cpu().numpy())
        labels.append(batch["labels"].cpu().numpy())

    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    return f1.compute(predictions=predictions, references=labels, average='macro')

In [7]:
def pretrain(raw_dataset, tokenizer, transform_raw, fp16=True):
    print("=== Pretrain ===")

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=CFG.mlm_probability
    )

    tokenized_data = raw_dataset.with_transform(transform_raw)

    train_dataset = tokenized_data["train"]

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
        collate_fn=data_collator,
    )

    model = BertForMaskedLM.from_pretrained(
        CFG.base_model_name
    ).cuda()

    num_training_steps = CFG.pretrain_epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    for i in range(CFG.pretrain_epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer, fp16)
        print(f'Epoch {i+1}')

    model_path = f"{CFG.name}_pretrain"
    model.save_pretrained(model_path)
    return model_path

In [8]:
def finetine(base_model, train_dataset, eval_dataset, device, fp16=False):
    print("=== Finetune ===")
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model, num_labels=CFG.num_classes
    ).cuda()

    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )

    eval_loader = DataLoader(
        eval_dataset,
        batch_size=CFG.eval_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=False,
    )

    num_training_steps = CFG.classification_epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    model.to(device)
    for i in range(CFG.classification_epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer)
        accuracy = evaluate_model(model, eval_loader)
        print(f'Epoch {i+1} {accuracy}')

    model_path = f"{CFG.name}_finetune"
    model.save_pretrained(model_path)
    return model_path

In [9]:
def main():

    if CFG.if_wandb:
        wandb.init(
            name=CFG.name,
            project=CFG.project,
            config=cfg
        )

    raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)
    classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Train tokenizer to Lang X
    tokenizer = train_tokenizer(raw_dataset)

    # for raw set
    def transform_raw(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        return inputs

    # for problem set
    def transform(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_devanagari(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        inputs["labels"] = example_batch["label"]
        return inputs

    tokenized_data = classification_dataset.with_transform(transform)

    train_dataset = tokenized_data["train"]
    eval_dataset = tokenized_data["dev"]

    # Continual Pre-Training of MLM
    if CFG.pretrained_model_name is None:
        pretrained_model_path = pretrain(raw_dataset, tokenizer, transform_raw, fp16=True)
    else:
        pretrained_model_path = CFG.pretrained_model_name

    # Finetune with normal dataset
    finetuned_model_path = finetine(pretrained_model_path, train_dataset, eval_dataset, device)

    # Get pseudo label
    pseudo_data, confidences = pseudo_get_data(raw_dataset, transform_raw, finetuned_model_path, device)
    pseudo_labeled_tokens = pseudo_data.with_transform(transform)

    combined_train_dataset = concatenate_datasets([pseudo_labeled_tokens, train_dataset])

    # Finetune with pseudo dataset and normal dataset
    final_model_name = finetine(pretrained_model_path, combined_train_dataset, eval_dataset, device)

    return final_model_name, tokenizer

In [10]:
final_model_name, tokenizer = main()

  0%|          | 0/611245 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


=== Finetune ===


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

Epoch 28 {'f1': 0.7789841855115814}


  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 29 {'f1': 0.7826439221176296}


  0%|          | 0/94 [00:00<?, ?it/s]

Epoch 30 {'f1': 0.7829789963123297}


In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    final_model_name, num_labels=CFG.num_classes
)
model.push_to_hub(
    f"ioai2024japan/{CFG.name}",
    token=userdata.get('hf_write'), private=True
)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/chizu_000_task2_complete_all/commit/f529937ebbe1abd4fec5898a3e1ca55130814196', commit_message='Upload BertForSequenceClassification', commit_description='', oid='f529937ebbe1abd4fec5898a3e1ca55130814196', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
tokenizer.push_to_hub(
    f"ioai2024japan/{CFG.name}",
    token=userdata.get('hf_write'), private=True
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/chizu_000_task2_complete_all/commit/36c66011a4332c4bc26b86547a3f0a4503744a11', commit_message='Upload tokenizer', commit_description='', oid='36c66011a4332c4bc26b86547a3f0a4503744a11', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
def terminate_session():
    # Terminate this session

    from google.colab import runtime
    runtime.unassign()

terminate_session()